# The JuMP ecosystem for mathematical optimization: Topics

This notebook is part of a workshop at [JuliaCon 2018](http://juliacon.org/2018/) and is based on materials and notebooks from various sources including the [JuliaOpt notebooks](https://github.com/JuliaOpt/juliaopt-notebooks), the [2018 ISCO Spring School](https://github.com/joehuchette/ISCO-spring-school) and the [second annual JuMP-dev workshop](http://www.juliaopt.org/meetings/bordeaux2018/).

## Disclaimer
This notebook is only working under the versions:

- JuMP 0.19 (unreleased, but currently in master)

- MathOptInterface 0.4.1

- GLPK 0.6.0

In [2]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

## Duality

I can get duals from constraint references.

In [3]:
model = Model(optimizer = GLPK.GLPKOptimizerLP())
@variable(model, x >= 0)
@variable(model, y >= 0)
@constraint(model,inequality, x + y <= 1)         
@objective(model, Max, x + 2y)
JuMP.optimize(model)
@show JuMP.hasresultvalues(model)
@show JuMP.terminationstatus(model) == MOI.Success
@show JuMP.primalstatus(model) == MOI.FeasiblePoint
@show JuMP.dualstatus(model) == MOI.FeasiblePoint
@show JuMP.resultdual(inequality)

JuMP.hasresultvalues(model) = true
JuMP.terminationstatus(model) == MOI.Success = true
JuMP.primalstatus(model) == MOI.FeasiblePoint = true
JuMP.dualstatus(model) == MOI.FeasiblePoint = true


-2.0

JuMP.resultdual(inequality) = -2.0


I can also get "duals" for variable bounds.

In [4]:
@show JuMP.resultdual(JuMP.LowerBoundRef(x))
@show JuMP.resultdual(JuMP.LowerBoundRef(y))

0.0

JuMP.resultdual(JuMP.LowerBoundRef(x)) = 1.0
JuMP.resultdual(JuMP.LowerBoundRef(y)) = 0.0


For sign conventions and precise definition of "dual" problem see [Duality](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual.html#Duals-1) in [MOI Manual](http://www.juliaopt.org/MathOptInterface.jl/stable)